# DLT Notebook - Gold Layer

In [0]:
import dlt

In [0]:
looktables_rules = {
    "rule1": "show_id is NOT NULL"
}

In [0]:
# Defining a streaming table to ingest data from a volume

@dlt.table(
    name = "gold_netflixdirectors"
)

@dlt.expectAll_or_drop(looktables_rules)

def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprjdlt.dfs.core.windows.net/netflix_directors")
    return df


In [0]:
@dlt.table(
    name = "gold_netflixcast"
)

@dlt.expectAll_or_drop(looktables_rules)

def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprjdlt.dfs.core.windows.net/netflix_cast")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcountries"
)
@dlt.expectAll_or_drop(looktables_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprjdlt.dfs.core.windows.net/netflix_countries")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcategory"
)
@dlt.expect_or_drop("rule1","show_id is NOT NULL")
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@netflixprjdlt.dfs.core.windows.net/netflix_category")
    return df

In [0]:
@dlt.table
def gold_stg_netflixtitles():
     df = spark.readStream.format("delta").load("abfss://silver@netflixprjdlt.dfs.core.windows.net/netflix_titles")
     return df


In [0]:
from pyspark.sql.functions import *

In [0]:
@dlt.view
def gold_trns_netflixtitles():
    df = spark.readStream.table("LIVE.gold_stg_netflixtitles")
    df = df.withColumn("newflag",lit(1))
    return df

In [0]:
masterdata_rules = {
    "rule1" : "newflag is NOT NULL",
    "rule2" : "show_id is NOT NULL"
}

In [0]:
@dlt.table
@dlt.expect_all_or_drop(masterdata_rules)
def gold_netflix_titles():
    df = spark.readStream.table("LIVE.gold_trns_netflixtitles")
    return df